In [521]:
from music21 import *

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

In [522]:
# get bach paths
paths = corpus.getComposer('bach')

paths = paths[:2]
notes = []
midi_notes = []
for path in paths:
    score = converter.parse(paths[0])

    # get the first part
    part = score.parts[0]

    # get all notes from the part
    notes.extend(list(map(lambda note: note, part.recurse().notes)))

midi_notes = list(map(lambda note: note.pitch.midi, notes))

print(f'len(notes): {len(notes)}')
print(f'len(midi_notes): {len(midi_notes)}')
print(midi_notes)

len(notes): 306
len(midi_notes): 306
[65, 67, 60, 65, 53, 57, 53, 57, 60, 65, 70, 69, 67, 65, 67, 69, 69, 65, 67, 69, 70, 72, 70, 69, 67, 67, 72, 72, 71, 69, 71, 72, 65, 65, 70, 69, 67, 65, 60, 65, 60, 60, 57, 53, 60, 60, 60, 57, 65, 67, 60, 65, 53, 57, 53, 57, 60, 65, 70, 69, 67, 65, 67, 69, 69, 65, 67, 69, 70, 72, 70, 69, 67, 67, 72, 72, 71, 69, 71, 72, 65, 65, 70, 69, 67, 65, 60, 65, 60, 60, 57, 53, 60, 60, 60, 57, 67, 60, 60, 60, 60, 60, 60, 65, 67, 65, 67, 69, 67, 65, 65, 60, 65, 60, 72, 70, 69, 70, 72, 70, 69, 67, 65, 60, 72, 60, 67, 65, 64, 65, 65, 53, 60, 60, 60, 57, 69, 67, 69, 70, 72, 72, 70, 69, 69, 67, 65, 65, 53, 60, 60, 60, 57, 65, 67, 60, 65, 53, 57, 53, 57, 60, 65, 70, 69, 67, 65, 67, 69, 69, 65, 67, 69, 70, 72, 70, 69, 67, 67, 72, 72, 71, 69, 71, 72, 65, 65, 70, 69, 67, 65, 60, 65, 60, 60, 57, 53, 60, 60, 60, 57, 65, 67, 60, 65, 53, 57, 53, 57, 60, 65, 70, 69, 67, 65, 67, 69, 69, 65, 67, 69, 70, 72, 70, 69, 67, 67, 72, 72, 71, 69, 71, 72, 65, 65, 70, 69, 67, 65, 60, 65

In [523]:
# turn notes in to training sequences of 5
# map each sequence to note that follows
# [65, 67, 60, 65, 53] -> 57
# [67, 60, 65, 53, 57] -> 53

seq_length = 5

print(f'len(midi_notes): {len(midi_notes)}')

data = []
labels = []
for i in range(len(midi_notes)):
    seq_end = i + seq_length
    
    if seq_end >= len(midi_notes): break
    
#     print(f'i: {i}')
#     print(f'seq_end: {seq_end}')
    
    seq = midi_notes[i:seq_end]        
    data.append(seq)
    labels.append(midi_notes[seq_end])

print(f'data:\n{data[:3]}')
print(f'labels:\n{labels[:3]}')

len(midi_notes): 306
data:
[[65, 67, 60, 65, 53], [67, 60, 65, 53, 57], [60, 65, 53, 57, 53]]
labels:
[57, 53, 57]


In [524]:
# # normalize
# from sklearn.preprocessing import MinMaxScaler

# scaler_data = MinMaxScaler(feature_range=(0, 1))
# scaler_data = scaler.fit(data)
# data_norm = scaler_data.transform(data)

# scaler_labels = MinMaxScaler(feature_range=(0, 1))
# scaler_labels = scaler.fit(labels)
# labels_norm = scaler_labels.transform(labels)

# print(data_norm.shape)
# print(labels_norm.shape)

# for i, seq in enumerate(data_norm):
#     print(f'{seq} -> {labels_norm[i]}')

In [ ]:
# get the data in shape
data = np.array(data, dtype=float)
data = data.reshape(-1, 5, 1)
data = data / 128

labels = np.array(labels, dtype=float)
labels = labels.reshape(-1, 1)
labels = labels / 128

print(data.shape)
print(labels.shape)

for i, seq in enumerate(data):
    print(f'{seq} -> {labels[i]}')

(301, 5, 1)
(301, 1)
[[0.5078125]
 [0.5234375]
 [0.46875  ]
 [0.5078125]
 [0.4140625]] -> [0.4453125]
[[0.5234375]
 [0.46875  ]
 [0.5078125]
 [0.4140625]
 [0.4453125]] -> [0.4140625]
[[0.46875  ]
 [0.5078125]
 [0.4140625]
 [0.4453125]
 [0.4140625]] -> [0.4453125]
[[0.5078125]
 [0.4140625]
 [0.4453125]
 [0.4140625]
 [0.4453125]] -> [0.46875]
[[0.4140625]
 [0.4453125]
 [0.4140625]
 [0.4453125]
 [0.46875  ]] -> [0.5078125]
[[0.4453125]
 [0.4140625]
 [0.4453125]
 [0.46875  ]
 [0.5078125]] -> [0.546875]
[[0.4140625]
 [0.4453125]
 [0.46875  ]
 [0.5078125]
 [0.546875 ]] -> [0.5390625]
[[0.4453125]
 [0.46875  ]
 [0.5078125]
 [0.546875 ]
 [0.5390625]] -> [0.5234375]
[[0.46875  ]
 [0.5078125]
 [0.546875 ]
 [0.5390625]
 [0.5234375]] -> [0.5078125]
[[0.5078125]
 [0.546875 ]
 [0.5390625]
 [0.5234375]
 [0.5078125]] -> [0.5234375]
[[0.546875 ]
 [0.5390625]
 [0.5234375]
 [0.5078125]
 [0.5234375]] -> [0.5390625]
[[0.5390625]
 [0.5234375]
 [0.5078125]
 [0.5234375]
 [0.5390625]] -> [0.5390625]
[[0.523437

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=4)

In [ ]:
model = Sequential()
model.add(LSTM((1), batch_input_shape=(None,5, 1), return_sequences=False))
# model.add(Dropout(0.2))
# model.add(LSTM((1), return_sequences=False))
# model.add(Dense(1, activation='sigmoid'))
# model.add(LSTM((1), return_sequences=False))
# model.add(Dense(1, activation='relu'))


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print(model.input_shape)
print(model.output_shape)
print(model.summary())

(None, 5, 1)
(None, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_69 (LSTM)               (None, 1)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_test, y_test))

Train on 240 samples, validate on 61 samples
Epoch 1/200
240/240 [==============================] - 6s 24ms/step - loss: 0.2427 - acc: 0.0000e+00 - val_loss: 0.2369 - val_acc: 0.0000e+00
Epoch 2/200
240/240 [==============================] - 0s 432us/step - loss: 0.2314 - acc: 0.0000e+00 - val_loss: 0.2253 - val_acc: 0.0000e+00
Epoch 3/200
240/240 [==============================] - 0s 466us/step - loss: 0.2196 - acc: 0.0000e+00 - val_loss: 0.2131 - val_acc: 0.0000e+00
Epoch 4/200
240/240 [==============================] - 0s 450us/step - loss: 0.2071 - acc: 0.0000e+00 - val_loss: 0.2004 - val_acc: 0.0000e+00
Epoch 5/200
240/240 [==============================] - 0s 464us/step - loss: 0.1942 - acc: 0.0000e+00 - val_loss: 0.1872 - val_acc: 0.0000e+00
Epoch 6/200
240/240 [==============================] - 0s 449us/step - loss: 0.1807 - acc: 0.0000e+00 - val_loss: 0.1735 - val_acc: 0.0000e+00
Epoch 7/200
240/240 [==============================] - 0s 464us/step - loss: 0.1668 - acc: 0.0000e

240/240 [==============================] - 0s 481us/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 58/200
240/240 [==============================] - 0s 443us/step - loss: 0.0015 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 59/200
240/240 [==============================] - 0s 418us/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 60/200
240/240 [==============================] - 0s 446us/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 61/200
240/240 [==============================] - 0s 451us/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 62/200
240/240 [==============================] - 0s 455us/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch 63/200
240/240 [==============================] - 0s 463us/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0014 - val_acc: 0.0000e+00
Epoch

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=16)
print(score)

In [ ]:
results= model.predict(x_test)
# results = results * 128
# y_test = y_test * 128
print(f'results:\n{results}')
print(f'y_test:\n{y_test}')

In [ ]:
plt.scatter(range(len(results)), results, c='r')
plt.scatter(range(len(y_test)), y_test, c='g')

In [ ]:
plt.plot(history.history['loss'])